In [12]:
%reset

import sympy as sym
import numpy as np
import pickle as pickle

sym.init_printing()
from IPython.display import display

#symbols
legs = ['A','B']
links = ['UL','LL','UR','LR'] #upper left link, lower left link etc etc
bases = ['body','boom']

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [13]:
#Parameters
masses = {}
lengths = {}
inertias = {}
COMR = {} # COM ratio distance

BLD = sym.symbols('BLD') # from COM of body leg distance
g = sym.symbols('g')
foot_angle = sym.symbols('foot_angle') # 136*(np.pi/180)
foot_length = sym.symbols('foot_length')
cg = sym.symbols('cg')
lx_boom = sym.symbols('lx_boom')
ly_boom = sym.symbols('ly_boom')
y_bodyOffset = sym.symbols('y_bodyOffset')
y_boomOffset = sym.symbols('y_boomOffset')
x_footOffset = sym.symbols('x_footOffset')

for leg in legs:
    for link in links:   
        masses.update({(link,leg): sym.symbols('m_{0}'.format(link+leg))})
        lengths.update({(link,leg): sym.symbols('l_{0}'.format(link+leg))})
        inertias.update({(link,leg): sym.symbols('In_{0}'.format(link+leg))})
        
for base in bases:
    masses.update({(base): sym.symbols('m_{0}'.format(base))})
    lengths.update({(base): sym.symbols('l_{0}'.format(base))})
    if base == 'boom':
        inertias.update({(base,'X'): sym.symbols('In_{0}_X'.format(base))})
        inertias.update({(base,'Y'): sym.symbols('In_{0}_Y'.format(base))})
    else:
        inertias.update({(base): sym.symbols('In_{0}'.format(base))})

#gen coordinates
x_body = 0.0
y_body = 1.0
dx_body = 0.0
dy_body = 0.0
ddx_body = 0.0
ddy_body = 0.0

th = {}
dth = {}
ddth = {}

for leg in legs:
    for link in links:
        th.update({(link,leg): sym.symbols('theta_{0}'.format(link+leg))}) 
        dth.update({(link,leg): sym.symbols('\dot{{\\theta}}_{{{0}}}'.format(link+leg))})
        ddth.update({(link,leg): sym.symbols('\ddot{{\\theta}}_{{{0}}}'.format(link+leg))})

In [14]:
q = sym.Matrix([th[link,leg] for leg in legs for link in links])
dq = sym.Matrix([dth[link,leg] for leg in legs for link in links])
ddq = sym.Matrix([ddth[link,leg] for leg in legs for link in links])

In [15]:
q,dq,ddq

⎛⎡θ_ULA⎤  ⎡\dot{\theta}_{ULA}⎤  ⎡\ddot{\theta}_{ULA}⎤⎞
⎜⎢     ⎥  ⎢                  ⎥  ⎢                   ⎥⎟
⎜⎢θ_LLA⎥  ⎢\dot{\theta}_{LLA}⎥  ⎢\ddot{\theta}_{LLA}⎥⎟
⎜⎢     ⎥  ⎢                  ⎥  ⎢                   ⎥⎟
⎜⎢θ_URA⎥  ⎢\dot{\theta}_{URA}⎥  ⎢\ddot{\theta}_{URA}⎥⎟
⎜⎢     ⎥  ⎢                  ⎥  ⎢                   ⎥⎟
⎜⎢θ_LRA⎥  ⎢\dot{\theta}_{LRA}⎥  ⎢\ddot{\theta}_{LRA}⎥⎟
⎜⎢     ⎥, ⎢                  ⎥, ⎢                   ⎥⎟
⎜⎢θ_ULB⎥  ⎢\dot{\theta}_{ULB}⎥  ⎢\ddot{\theta}_{ULB}⎥⎟
⎜⎢     ⎥  ⎢                  ⎥  ⎢                   ⎥⎟
⎜⎢θ_LLB⎥  ⎢\dot{\theta}_{LLB}⎥  ⎢\ddot{\theta}_{LLB}⎥⎟
⎜⎢     ⎥  ⎢                  ⎥  ⎢                   ⎥⎟
⎜⎢θ_URB⎥  ⎢\dot{\theta}_{URB}⎥  ⎢\ddot{\theta}_{URB}⎥⎟
⎜⎢     ⎥  ⎢                  ⎥  ⎢                   ⎥⎟
⎝⎣θ_LRB⎦  ⎣\dot{\theta}_{LRB}⎦  ⎣\ddot{\theta}_{LRB}⎦⎠

In [16]:
for leg in legs:
    for link in links:
        COMR.update({(link,leg): sym.symbols('COMR_{0}'.format(link+leg))})

In [17]:
''' Setup positions (to COM) and velocities'''

r_bases = {}

# position to COM of each link - where the mass is approximated to be
r_bases.update({'body': sym.Matrix([[x_body,
                                     y_body]]).transpose()})

r_bases.update({'boom': sym.Matrix([[(cg/lx_boom)*x_body,
                                     (cg/ly_boom)*y_body]]).transpose()})

# define absolute angles
abs_angles = {}
r_links = {}

# using small angle approximation
abs_angles.update({('boom','thx_boom'): x_body/lx_boom})
abs_angles.update({('boom','thy_boom'): y_body/ly_boom})

for leg in legs:
    
    abs_angles.update({('LU',leg): th['UL',leg]})
    abs_angles.update({('LL',leg): th['LL',leg] - (np.pi - abs_angles['LU',leg])})
    abs_angles.update({('RU',leg): th['UR',leg]})
    abs_angles.update({('RL',leg): th['LR',leg] - (np.pi - abs_angles['RU',leg]) }) 
    
    r_links.update({('UL',leg): sym.Matrix([[x_body-BLD-lengths['UL',leg]*COMR['UL',leg]*sym.cos(abs_angles['LU',leg]),
                                             y_body-y_bodyOffset-lengths['UL',leg]*COMR['UL',leg]*sym.sin(abs_angles['LU',leg])]]).transpose()})
    
    r_links.update({('UR',leg): sym.Matrix([[x_body+BLD+lengths['UR',leg]*COMR['UR',leg]*sym.cos(np.pi-abs_angles['RU',leg]),
                                             y_body-y_bodyOffset-lengths['UR',leg]*COMR['UR',leg]*sym.sin(np.pi-abs_angles['RU',leg])]]).transpose()})
    
    r_links.update({('LL',leg): sym.Matrix([[r_links['UL',leg][0]-(1-COMR['UL',leg])*lengths['UL',leg]*sym.cos(abs_angles['LU',leg])+lengths['LL',leg]*COMR['LL',leg]*sym.cos(np.pi-abs_angles['LL',leg]),
                                             r_links['UL',leg][1]-(1-COMR['UL',leg])*lengths['UL',leg]*sym.sin(abs_angles['LU',leg])-lengths['LL',leg]*COMR['LL',leg]*sym.sin(np.pi-abs_angles['LL',leg])]]).transpose()})

    r_links.update({('LR',leg): sym.Matrix([[r_links['UR',leg][0]+(1-COMR['UR',leg])*lengths['UR',leg]*sym.cos(np.pi-abs_angles['RU',leg])-lengths['LR',leg]*COMR['LR',leg]*sym.cos(abs_angles['RL',leg]) + x_footOffset,
                                             r_links['UR',leg][1]-(1-COMR['UR',leg])*lengths['UR',leg]*sym.sin(np.pi-abs_angles['RU',leg])-lengths['LR',leg]*COMR['LR',leg]*sym.sin(abs_angles['RL',leg])]]).transpose()})

In [18]:
dr_bases = {}
dr_links = {}
w_links = {}
w_bases = {}



thx_boom = sym.Matrix([[sym.asin(x_body/lx_boom)]])
thy_boom = sym.Matrix([[sym.asin(y_body/ly_boom)]])
dthx_boom = thx_boom.jacobian(q)*dq
dthy_boom = thy_boom.jacobian(q)*dq

w_bases.update({('boom','dthx_boom'): dthx_boom[0]})
w_bases.update({('boom','dthy_boom'): dthy_boom[0]})

# w_bases.update({('boom','dthx_boom'): dx_body/lx_boom})
# w_bases.update({('boom','dthy_boom'): dy_body/ly_boom})

for base in bases:
    dr_bases.update({base: r_bases[base].jacobian(q)*dq})

for leg in legs:
    
    w_links.update({('UL',leg): dth['UL',leg]})
    w_links.update({('LL',leg): dth['UL',leg] + dth['LL',leg]})
    w_links.update({('UR',leg): dth['UR',leg]})
    w_links.update({('LR',leg): dth['UR',leg] + dth['LR',leg]})
    
    for link in links:
        dr_links.update({(link,leg): r_links[link,leg].jacobian(q)*dq})
    

In [19]:
T_bases = {}
T_links = {}
V_bases = {}
V_links = {}

# Kinetic Energy
T_bases.update({'body': 0.5*sym.Matrix([[masses['body'],masses['body']]])*sym.matrix_multiply_elementwise(dr_bases['body'],dr_bases['body'])})
T_bases.update({'boom': 0.5*sym.Matrix([[masses['boom'],masses['boom']]])*sym.matrix_multiply_elementwise(dr_bases['boom'],dr_bases['boom']) + sym.Matrix([[0.5*w_bases['boom','dthx_boom']*w_bases['boom','dthx_boom']*inertias['boom','X']]]) + sym.Matrix([[0.5*w_bases['boom','dthy_boom']*w_bases['boom','dthy_boom']*inertias['boom','Y']]])})

# Potential Energy
for base in bases:
    if base == 'body':
        V_bases.update({base: masses[base]*g*r_bases[base][1]})
    else:
        V_bases.update({base: masses[base]*g*(y_boomOffset + ((y_body-y_boomOffset)*cg/ly_boom))})

for leg in legs:
    for link in links:
        T_links.update({(link,leg): 0.5*sym.Matrix([[masses[link,leg],masses[link,leg]]])*sym.matrix_multiply_elementwise(dr_links[link,leg],dr_links[link,leg]) + sym.Matrix([[0.5*w_links[link,leg]*w_links[link,leg]*inertias[link,leg]]])})
        V_links.update({(link,leg): masses[link,leg]*g*r_links[link,leg][1]})

Ttot = T_bases['body'] + T_bases['boom']
Vtot = V_bases['body'] + V_bases['boom']
    
for leg in legs:
    for link in links:
        Ttot = Ttot + T_links[link,leg] 
        Vtot = Vtot + V_links[link,leg] 
        
Ttot = Ttot[0]

In [20]:
# Manipulator Equation (M*ddq + C + G = Tau + Q)

# M Matrix
M = sym.zeros(len(q),len(q))

for i in range(len(q)):
    for j in range(len(q)):
        M[i,j] = sym.diff(sym.diff(Ttot,dq[i]),dq[j])

M = sym.simplify(M)

M_size = int(np.sqrt(len(M))) # assumes square matrix (len(M) = 4x4 = 16)

dM = sym.zeros(M_size,M_size)

for i in range(M_size):
    for j in range(M_size):
        dM[i,j] = sym.Matrix([M[i,j]]).jacobian(q)*dq

# C Matrix
C = dM*dq - (sym.Matrix([Ttot]).jacobian(q)).transpose() # 4x1 - 1x4^Transpose

# G Matrix
G = (sym.Matrix([Vtot]).jacobian(q)).transpose() # 4x1

tau = {}
lamda = {}
GRF = {}

connect_position = {}
foot_position = {}

Jbn = {}
Jfoot = {}
lamda_GRF = {}
lamda_connect = {}

Tau = sym.Matrix([])

Q = sym.zeros(len(q),1)

for leg in legs:
    tau.update({('left',leg): sym.symbols('tau_left{0}'.format(leg))})
    tau.update({('right',leg): sym.symbols('tau_right{0}'.format(leg))})
    
    lamda.update({('x',leg): sym.symbols('lamda_x{0}'.format(leg))})
    lamda.update({('y',leg): sym.symbols('lamda_y{0}'.format(leg))})
    
    GRF.update({('x',leg): sym.symbols('GRF_x{0}'.format(leg))})
    GRF.update({('y',leg): sym.symbols('GRF_y{0}'.format(leg))})
    
    
    connect_position.update({('left',leg): sym.Matrix([[r_links['LL',leg][0]+(1-COMR['LL',leg])*lengths['LL',leg]*sym.cos(np.pi-abs_angles['LL',leg])],
                                                       [r_links['LL',leg][1]-(1-COMR['LL',leg])*lengths['LL',leg]*sym.sin(np.pi-abs_angles['LL',leg])]]).transpose()})
    
    connect_position.update({('right',leg): sym.Matrix([[r_links['LR',leg][0]-(1-COMR['LR',leg])*lengths['LR',leg]*sym.cos(abs_angles['RL',leg]) - x_footOffset],
                                                        [r_links['LR',leg][1]-(1-COMR['LR',leg])*lengths['LR',leg]*sym.sin(abs_angles['RL',leg])]]).transpose()})

    foot_position.update({('right',leg): sym.Matrix([[connect_position['right',leg][0] + foot_length*sym.cos(foot_angle - (np.pi)/2)],
                                                     [connect_position['right',leg][1] - foot_length*sym.sin(foot_angle - (np.pi)/2)]]).transpose()})

    Jbn.update({('left',leg): connect_position['left',leg].jacobian(q)})                    
    Jbn.update({('right',leg): connect_position['right',leg].jacobian(q)}) 
    
    Jfoot.update({(leg): foot_position['right',leg].jacobian(q)})
    
    lamda_GRF.update({(leg): sym.Matrix([[GRF['x',leg]],[GRF['y',leg]]])})
    
    lamda_connect.update({('left',leg): sym.Matrix([[-lamda['x',leg]],[-lamda['y',leg]]])})
    lamda_connect.update({('right',leg): sym.Matrix([[lamda['x',leg]],[lamda['y',leg]]])})
    
    Tau = Tau.col_join(sym.Matrix([[tau['left',leg]],[0],[tau['right',leg]],[0]]))
    
    Q = Q + (Jfoot[leg].transpose()*lamda_GRF[leg]) + (Jbn['left',leg].transpose()*lamda_connect['left',leg]) + (Jbn['right',leg].transpose()*lamda_connect['right',leg])

In [21]:
len(Tau)

In [22]:
# Manipulator Equation 
EOM = M*ddq + G + C - Tau - Q 

# save the EOM in a file (saving the EOM saves lots of time)
import dill
dill.dump_session('unsimplified_bipod_thbodlocked_v3_no_xy.db')

In [23]:
# with open('mark_unsimplified_bipod_v3','w') as f:
#     for j in range(len(q)):
#         f.write(str(EOM[j]))
#         f.write('\n')


In [24]:
# import dill
# dill.load_session('unsimplified_bipod_thbodlocked_v3.db')

In [25]:

EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()
    print('j = {0} is done'.format(j+1))
    dill.dump_session('simplified_bipodv3_thbodlocked_no_xy_EOM_{0}.db'.format(j+1))

j = 1 is done
j = 2 is done
j = 3 is done
j = 4 is done
j = 5 is done
j = 6 is done
j = 7 is done
j = 8 is done


In [26]:
import dill
dill.dump_session('simplified_bipod_thbodlocked_no_xy_final_mark.db')